In [1]:
data_type = {
    2: '水位',
    6: '流量'
}

In [2]:
import csv
import time
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

#時間を、文字列←→日付型に相互に変換。どちらへ変換かは型で判別。
def time_tyconv(time):
    if isinstance(time,str):
        return datetime.strptime(time,'%Y%m%d')
    elif isinstance(time,datetime):
        return time.strftime("%Y%m%d")
    else:
        raise TypeError('type-err!')

#1ヶ月後の日付データを求める
def date_after_days(time):
    day_after = time_tyconv(time) + relativedelta(months=1)
    return time_tyconv(day_after)


#任意期間のデータを水水DBからスクレイピングする関数。
def scrp_DB(start_date, location_id, result_list):
    base_url = 'http://www1.river.go.jp'
    url = "http://www1.river.go.jp/cgi-bin/DspWaterData.exe"
    params = {
        "KIND":2,
        "ID":location_id,
        "BGNDATE":start_date,
        "ENDDATE":"20990430",
        "KAWABOU":"NO"
    }
    #iframeのhtmlからsrcURLを取得
    resp = requests.get(url,params=params)
    soup = BeautifulSoup(resp.text, 'html.parser')
    if soup.find('tbody') is None:
        return
    tbody = soup.find('tbody').find_all('tr')

    for i in range(len(tbody)):
        if i < 2 or i == len(tbody) - 1:
            continue
        result_row = []
        date = tbody[i].find('th').get_text()
        th_list = tbody[i].find_all('td')
        for time, td in enumerate(th_list):
            cell = td.get_text()
            try:
                cell = float(cell)
            except ValueError as e:
                cell = 0.0
            result_list.append([
                datetime.strptime(date + ' ' + str(time) + ':00:00', "%Y/%m/%d %H:%M:%S"),
                cell
            ])

    return result_list

In [1]:
#取得期間、地点番号（おそらく４から始まるほう）、保存ファイル名を設定
start_date = "20190401"
csvfile_name = "result_johoku.csv"
location_id = "302041282204020"
n_step = 24
type_no = 2
type_name = data_type[type_no]

result = []

for n in range(n_step):
    result = scrp_DB(start_date, location_id, result)
    #start_dateに1ヶ月、次のループへ
    start_date = date_after_days(start_date)
    #サーバーへの負荷を考慮し、2秒置く
    time.sleep(2)
files_df = pd.DataFrame(result, columns = ['時刻', type_name])

#CSVとして出力
files_df.to_csv(csvfile_name, index = False)

# グラフの描画
plt.plot(files_df['時刻'], files_df[type_name])
plt.show()